In [1]:
import os
import sys
import glob
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import mrcc
from mrcc.p_from_scaled_containment import compute_confidence_intervals

sns.set_context("paper")

In [2]:
pd.__version__

'0.24.2'

## Import Simulated Read Data

In [3]:
%%bash
mkdir -p ../data
curl -L https://osf.io/xn7vt/download -o ../data/simreads-compare.dnainput.csv.gz
ls ../data

simreads-compare.dnainput.csv.gz
simreads-compare.dnainput.mrcc.csv.gz
simreads-compare.dnainput.processed.csv.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   459  100   459    0     0    746      0 --:--:-- --:--:-- --:--:--   746
100 7817k  100 7817k    0     0  2370k      0  0:00:03  0:00:03 --:--:-- 3156k


In [4]:
infoDF = pd.read_csv("../data/simreads-compare.dnainput.csv.gz")
infoDF.head()

,comparison_name,sig1_name,sig2_name,alphabet,ksize,scaled,jaccard,max_containment,sig1_containment,sig2_containment,sig1_hashes,sig2_hashes,num_common,alpha-ksize
0,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,1,0.192024,0.329152,0.329152,0.315500,4974666,5189923,1637423,dna-21
1,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,100,0.193861,0.332344,0.332344,0.317521,49849,52176,16567,dna-21
2,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,1000,0.203267,0.343331,0.343331,0.332559,5001,5163,1717,dna-21
3,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,2000,0.203029,0.344026,0.344026,0.331274,2494,2590,858,dna-21
4,data-d0.05-f1-nogam-seed44,data-d0.05-f1-nogam-seed44-seq1,data-d0.05-f1-nogam-seed44-seq2,dna,21,1,0.192989,0.325068,0.322023,0.325068,4993000,4946231,1607860,dna-21


### Use `scaled=1` sketches to get an average number of unique k-mers for distance estimation.

We do not yet have the most accurate quantification of the total number of unique hashes in each sketch. 
To get around this for now, I sketched at scaled=1,100,1000,2000 and we can use the number of unique hashes/k-mers from the `scaled=1` sketches for all estimations. 
Here I create a `num_unique_kmers` column, which is the average number of k-mers/hashes in the `scaled=1` sketches for each comparison. 

In [5]:
sc1 = infoDF.loc[infoDF["scaled"]==1]
sc1.tail()

,comparison_name,sig1_name,sig2_name,alphabet,ksize,scaled,jaccard,max_containment,sig1_containment,sig2_containment,sig1_hashes,sig2_hashes,num_common,alpha-ksize
273580,data-d0.95-f3-gamma-seed427,data-d0.95-f3-gamma-seed427-seq1,data-d0.95-f3-gamma-seed427-seq2,dna,51,1,0.0,0.0,0.0,0.0,3557371,3626515,0,dna-51
273584,data-d0.95-f3-gamma-seed480,data-d0.95-f3-gamma-seed480-seq1,data-d0.95-f3-gamma-seed480-seq2,dna,51,1,0.0,0.0,0.0,0.0,3350727,3298352,0,dna-51
273588,data-d0.95-f3-gamma-seed576,data-d0.95-f3-gamma-seed576-seq1,data-d0.95-f3-gamma-seed576-seq2,dna,51,1,0.0,0.0,0.0,0.0,3362658,4249897,0,dna-51
273592,data-d0.95-f3-gamma-seed498,data-d0.95-f3-gamma-seed498-seq1,data-d0.95-f3-gamma-seed498-seq2,dna,51,1,0.0,0.0,0.0,0.0,3612682,3818313,0,dna-51
273596,data-d0.95-f3-gamma-seed550,data-d0.95-f3-gamma-seed550-seq1,data-d0.95-f3-gamma-seed550-seq2,dna,51,1,0.0,0.0,0.0,0.0,3365066,4281691,0,dna-51


In [6]:
# get a standardized num_unique k-mers per comparison (average between both sketches at scaled=1)
sc1["num_unique_kmers"] = (sc1["sig1_hashes"] + sc1["sig2_hashes"])/2 # do I need to round this??
sc1.head()

/Users/tessa/miniconda3/envs/jlab-mrcc/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,comparison_name,sig1_name,sig2_name,alphabet,ksize,scaled,jaccard,max_containment,sig1_containment,sig2_containment,sig1_hashes,sig2_hashes,num_common,alpha-ksize,num_unique_kmers
0,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,1,0.192024,0.329152,0.329152,0.315500,4974666,5189923,1637423,dna-21,5082294.5
4,data-d0.05-f1-nogam-seed44,data-d0.05-f1-nogam-seed44-seq1,data-d0.05-f1-nogam-seed44-seq2,dna,21,1,0.192989,0.325068,0.322023,0.325068,4993000,4946231,1607860,dna-21,4969615.5
8,data-d0.05-f1-nogam-seed64,data-d0.05-f1-nogam-seed64-seq1,data-d0.05-f1-nogam-seed64-seq2,dna,21,1,0.186960,0.315519,0.314529,0.315519,4976560,4960939,1565272,dna-21,4968749.5
12,data-d0.05-f1-nogam-seed54,data-d0.05-f1-nogam-seed54-seq1,data-d0.05-f1-nogam-seed54-seq2,dna,21,1,0.192410,0.323544,0.323544,0.321910,4991507,5016847,1614971,dna-21,5004177.0
16,data-d0.05-f1-nogam-seed148,data-d0.05-f1-nogam-seed148-seq1,data-d0.05-f1-nogam-seed148-seq2,dna,21,1,0.187846,0.316655,0.316655,0.315907,4994982,5006823,1581688,dna-21,5000902.5


In [7]:
# merge in the scaled=1 num_unique k-mers into the full dataframe
infoDF = pd.merge(infoDF, sc1[["comparison_name", "alpha-ksize", "num_unique_kmers"]], on=["comparison_name", "alpha-ksize"])
infoDF.head()

,comparison_name,sig1_name,sig2_name,alphabet,ksize,scaled,jaccard,max_containment,sig1_containment,sig2_containment,sig1_hashes,sig2_hashes,num_common,alpha-ksize,num_unique_kmers
0,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,1,0.192024,0.329152,0.329152,0.315500,4974666,5189923,1637423,dna-21,5082294.5
1,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,100,0.193861,0.332344,0.332344,0.317521,49849,52176,16567,dna-21,5082294.5
2,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,1000,0.203267,0.343331,0.343331,0.332559,5001,5163,1717,dna-21,5082294.5
3,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,2000,0.203029,0.344026,0.344026,0.331274,2494,2590,858,dna-21,5082294.5
4,data-d0.05-f1-nogam-seed44,data-d0.05-f1-nogam-seed44-seq1,data-d0.05-f1-nogam-seed44-seq2,dna,21,1,0.192989,0.325068,0.322023,0.325068,4993000,4946231,1607860,dna-21,4969615.5


In [8]:
# load fasta simulation information 
siminfo = pd.read_csv("../simreads-info.csv.gz")
# rename some useful columns
siminfo.rename(columns={"name": "comparison_name", "p-distance":"true p-distance"}, inplace=True)
siminfo.head()

,comparison_name,seed,freq(T),freq(C),freq(A),freq(G),rate(C-T),rate(A-T),rate(G-T),rate(A-C),rate(C-G),alpha,lgt1,lgt2,sites,core,true p-distance
0,data-d0.05-f1-gamma-seed1,1,0.25,0.25,0.25,0.25,1.57881,0.188961,0.184296,0.277635,0.571672,0.239,4997288,5120205,5532162,4585331,0.041891
1,data-d0.05-f1-gamma-seed2,2,0.25,0.25,0.25,0.25,1.30318,0.337478,0.282495,0.389976,0.857990,0.313,4978497,5080470,5532500,4526467,0.043604
2,data-d0.05-f1-gamma-seed3,3,0.25,0.25,0.25,0.25,1.67580,0.370299,0.254104,0.507523,0.281850,0.290,4973221,4744010,5261801,4455430,0.042999
3,data-d0.05-f1-gamma-seed4,4,0.25,0.25,0.25,0.25,1.96182,0.237414,0.177498,0.475090,0.293021,0.322,4979167,5068807,5394562,4653412,0.042863
4,data-d0.05-f1-gamma-seed5,5,0.25,0.25,0.25,0.25,1.60369,0.260495,0.183868,0.338577,0.185786,0.291,4987735,5023978,5329783,4681930,0.042532


In [9]:
# merge true p-distance info estimated distances dataframe
infoDF = pd.merge(infoDF, siminfo[["comparison_name", "true p-distance"]], on=["comparison_name"])
infoDF.head()

,comparison_name,sig1_name,sig2_name,alphabet,ksize,scaled,jaccard,max_containment,sig1_containment,sig2_containment,sig1_hashes,sig2_hashes,num_common,alpha-ksize,num_unique_kmers,true p-distance
0,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,1,0.192024,0.329152,0.329152,0.315500,4974666,5189923,1637423,dna-21,5082294.5,0.047855
1,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,100,0.193861,0.332344,0.332344,0.317521,49849,52176,16567,dna-21,5082294.5,0.047855
2,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,1000,0.203267,0.343331,0.343331,0.332559,5001,5163,1717,dna-21,5082294.5,0.047855
3,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,2000,0.203029,0.344026,0.344026,0.331274,2494,2590,858,dna-21,5082294.5,0.047855
4,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,31,1,0.108379,0.199794,0.199794,0.191507,4974660,5189923,993907,dna-31,5082291.5,0.047855


In [10]:
# replace any zeroes in jaccard, containment with np.nan to avoid errors
cols = ["jaccard", "max_containment", "sig1_containment", "sig2_containment"]
infoDF[cols].replace(['0', 0], np.nan, inplace=True)
infoDF.head()

/Users/tessa/miniconda3/envs/jlab-mrcc/lib/python3.6/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


,comparison_name,sig1_name,sig2_name,alphabet,ksize,scaled,jaccard,max_containment,sig1_containment,sig2_containment,sig1_hashes,sig2_hashes,num_common,alpha-ksize,num_unique_kmers,true p-distance
0,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,1,0.192024,0.329152,0.329152,0.315500,4974666,5189923,1637423,dna-21,5082294.5,0.047855
1,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,100,0.193861,0.332344,0.332344,0.317521,49849,52176,16567,dna-21,5082294.5,0.047855
2,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,1000,0.203267,0.343331,0.343331,0.332559,5001,5163,1717,dna-21,5082294.5,0.047855
3,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,21,2000,0.203029,0.344026,0.344026,0.331274,2494,2590,858,dna-21,5082294.5,0.047855
4,data-d0.05-f1-nogam-seed11,data-d0.05-f1-nogam-seed11-seq1,data-d0.05-f1-nogam-seed11-seq2,dna,31,1,0.108379,0.199794,0.199794,0.191507,4974660,5189923,993907,dna-31,5082291.5,0.047855


In [11]:
# save this csv.gz for use elsewhere
infoDF.to_csv("../data/simreads-compare.dnainput.processed.csv.gz", index=False)